## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is, essentially, the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

* Describe RDDs as fundamental storage units in the Spark computing environment
* Create RDDs from Python collections
* Set number of partitions for parallelizing RDDs
* Review an RDD's dependency graph at different stages of processing. 
* Apply the map(func) transformation to a given function on all elements of an RDD in different partitions
* Use collect() action to trigger the processing stage of spark's lazy evaluation
* Use count() action to calculate the number of elements of a parallelized RDD
* Use filter(func) to filter unwanted data from RDDs
* Develop an understanding of Python's lambda functions for RDDs processing


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies will performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified.
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the Database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new data set from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base data set (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the Driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some of key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [6]:
data = list(range(1,1001))
len(data)

#1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use 'local[*]' as the master.

In [7]:
import pyspark
sc = pyspark.SparkContext('local[*]')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-4-9ae820830206>:2 

Once you've created the SparkContext, you can use the `parallelize` method to create an rdd will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [8]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the getNumPartitions method.

In [9]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first n items in the RDD
* `top`: returns the top n items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [10]:
# count
rdd.count()

1000

In [11]:
# first
rdd.first()

1

In [14]:
# take
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [15]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [16]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 be a random number from 0-1.

In [17]:
import random
import numpy as np

nums = np.array(range(1,1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([4.14651878e-01, 1.37560200e+00, 4.32889544e-01, 1.60567095e+00,
       1.75193342e+00, 4.56997411e+00, 6.72927738e+00, 4.51259206e-01,
       5.03210911e+00, 2.88533230e+00, 7.52579786e+00, 1.02150680e+01,
       1.22671813e+01, 4.90243529e+00, 2.80274489e+00, 1.18119463e+01,
       1.69150138e+01, 8.71167956e+00, 4.87821800e+00, 1.82196409e+01,
       2.03868739e+01, 2.61482595e+00, 5.97682528e+00, 5.09997609e+00,
       2.30410331e+01, 1.15988069e+01, 2.35501361e+01, 2.28053263e+01,
       3.23445677e+00, 3.57770052e+00, 2.68335650e+01, 1.05933318e+01,
       3.04934426e+01, 1.38061787e+01, 2.37167736e+00, 1.27143720e+01,
       6.39074273e+00, 3.54994803e+01, 2.40265652e+01, 2.85115087e+00,
       2.42123642e+01, 3.93460391e+01, 2.97458909e+01, 2.69457545e+01,
       3.24417035e+01, 4.43346100e+01, 2.66300539e+01, 9.78293618e+00,
       3.04221187e+01, 3.15417330e+01, 1.72896944e+01, 2.86259692e+01,
       3.30669778e+01, 2.61390234e+01, 2.25308602e+01, 5.47644130e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [19]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.top(10)

[978.1711707037156,
 966.3790622928201,
 951.6103589642609,
 943.0260232255622,
 925.4540777796955,
 905.7946656812793,
 902.7128879322942,
 898.2305174989041,
 875.1522801376946,
 873.8356865046673]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax` that returns the amount of money our company will receive after the sale tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the map method and assign it to a variable `renenue_minus_tax`

In [20]:
def sales_tax(num):
    return num*.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [23]:
# perform action to retrieve rdd values
revenue_minus_tax.top(5)

[899.9174770474184,
 889.0687373093946,
 875.4815302471201,
 867.5839413675172,
 851.4177515573199]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a map method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [24]:
discounted = revenue_minus_tax.map(lambda x: x*.9)

In [25]:
discounted.take(10)

[0.3433317553672053,
 1.1389984537145734,
 0.35843254228830335,
 1.3294955459498712,
 1.4506008696672972,
 3.7839385624464157,
 5.571841667105927,
 0.3736426227300609,
 4.1665863394055975,
 2.38905514503997]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff use a collection method to see the 15 costliest items.

In [26]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[809.9257293426766,
 800.1618635784552,
 787.9333772224081,
 780.8255472307656,
 766.275976401588,
 749.9979831840993,
 747.4462712079396,
 743.7348684890926,
 724.6260879540112,
 723.5359484258645,
 717.513381609038,
 712.5907914457048,
 701.472180576738,
 701.042279888595,
 691.0092068646014]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [27]:
discounted.toDebugString()

b'(10) PythonRDD[13] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use flatMap rather than a simple map. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will a tuple with (original price, post-discount price).

In [29]:
mapped =price_items.map(lambda x: (x, x*0.92*0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.41465187846280827, 0.3433317553672053), (1.3756019972398228, 1.1389984537145734), (0.4328895438264533, 0.35843254228830335), (1.60567094921482, 1.3294955459498712), (1.751933417472581, 1.4506008696672972), (4.569974109234801, 3.7839385624464157), (6.729277375731796, 5.571841667105927), (0.4512592061957257, 0.3736426227300609), (5.0321091055623155, 4.1665863394055975), (2.8853323007729106, 2.38905514503997)]


Note that we have 1000 tuples created to our specification. Let's take a look at how flatMap differs in its implementation. Use the `flatMap` method with the same function you created above.

In [30]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92*0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.41465187846280827, 0.3433317553672053, 1.3756019972398228, 1.1389984537145734, 0.4328895438264533, 0.35843254228830335, 1.60567094921482, 1.3294955459498712, 1.751933417472581, 1.4506008696672972]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use flatmap rather than map in order to properly reduce to our specifications. This is not one of those instances, but int he upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the filter method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a filter function to meet the consultant's suggestion's specifications. set RDD = `selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [32]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)

# calculate total remaining in inventory 
selected_items.count()

267

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use a reduce method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [33]:
selected_items.reduce(lambda x,y :x + y)

124973.54205923277

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [34]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1,50),x))

sales_data.take(7)

[(10, 340.1372092660806),
 (14, 331.96495122764924),
 (34, 329.7311187594724),
 (38, 360.28794370346975),
 (42, 348.48779929455395),
 (17, 312.3777513609009),
 (19, 304.2086237124977)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called __reduceByKey__ to perform reducing operations while grouping by keys. After you have calculated the total, use the __sortBy__ method on the RDD to rank the users from the highest spending to the least spending.



In [37]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x,y :x + y)
total_spent.take(10)

[(50, 2807.4900040256034),
 (40, 2975.6054585062475),
 (20, 2168.622209853769),
 (30, 1815.8308508582518),
 (10, 3824.443331343592),
 (21, 4133.928715530867),
 (1, 2998.5187374541447),
 (11, 2791.1027779698156),
 (41, 2678.909607576481),
 (31, 2977.6604876055685)]

In [39]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(7, 5307.006359039091),
 (28, 4655.07190542527),
 (18, 4216.469706834092),
 (21, 4133.928715530867),
 (16, 3890.185176103345),
 (39, 3830.015259681435),
 (10, 3824.443331343592),
 (27, 3811.073310972455),
 (37, 3767.1658477982182),
 (14, 3444.9383453565933),
 (35, 3423.6732534063576),
 (49, 3296.8257039278888),
 (48, 3159.700079196664),
 (13, 3146.5470856769844),
 (1, 2998.5187374541447),
 (31, 2977.6604876055685),
 (40, 2975.6054585062475),
 (25, 2969.57105641926),
 (44, 2910.4984522570667),
 (50, 2807.4900040256034),
 (11, 2791.1027779698156),
 (8, 2746.022848886483),
 (41, 2678.909607576481),
 (4, 2574.6149309978146),
 (42, 2462.1472622153706),
 (24, 2412.8849578640843),
 (15, 2243.9468398981126),
 (29, 2176.021109581584),
 (20, 2168.622209853769),
 (3, 2102.897951558416),
 (43, 2066.5903983596822),
 (34, 2002.643144872046),
 (33, 1997.883639367078),
 (17, 1943.3047749632244),
 (5, 1940.9149914298632),
 (38, 1890.5193403699682),
 (47, 1883.2431394763826),
 (30, 1815.8308508582518),

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [42]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(6, 12),
 (44, 11),
 (10, 9),
 (27, 9),
 (47, 9),
 (49, 9),
 (1, 9),
 (11, 8),
 (22, 8),
 (40, 7),
 (50, 7),
 (38, 7),
 (43, 7),
 (34, 7),
 (15, 7),
 (9, 7),
 (2, 7),
 (42, 6),
 (28, 6),
 (25, 5),
 (7, 5),
 (19, 5),
 (26, 5),
 (35, 5),
 (21, 5),
 (3, 5),
 (31, 5),
 (4, 4),
 (45, 4),
 (32, 4),
 (13, 4),
 (5, 4),
 (18, 4),
 (46, 4),
 (8, 4),
 (36, 4),
 (20, 3),
 (30, 3),
 (23, 3),
 (12, 3),
 (37, 3),
 (33, 3),
 (16, 3),
 (39, 3),
 (48, 3),
 (17, 3),
 (41, 3),
 (24, 2),
 (29, 2),
 (14, 2)]

In [43]:
sc.stop()

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD, and extracting lineage and of an RDD in a spark application. We also used transformations and actions to perform calculations across RDDs on a distributed setup. Up next, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
